# Last-Mile TCT Replication: Retrieval

This notebook shows how to perform retrieval on indices using released TCT-ColBERT models.

Before you follow these examples, perform indexing (or download pre-built indices) using examples in tctrepro-lastmile-indexing.ipynb.

In [1]:
# Import packages
import pyterrier as pt
if not pt.started():
    pt.init()
from pyterrier.measures import *
from pyterrier_dr import NumpyIndex, TctColBert

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
# Build retrieval pipelines:
pipeline_v2 = (
    TctColBert('castorini/tct_colbert-v2-msmarco') >>
    NumpyIndex('indices/castorini__tct_colbert-v2-msmarco.np', verbose=True))

pipeline_v2_hn = (
    TctColBert('castorini/tct_colbert-v2-hn-msmarco') >>
    NumpyIndex('indices/castorini__tct_colbert-v2-hn-msmarco.np', verbose=True))

pipeline_v2_hnp = (
    TctColBert('castorini/tct_colbert-v2-hnp-msmarco') >>
    NumpyIndex('indices/castorini__tct_colbert-v2-hnp-msmarco.np', verbose=True))

In [3]:
# trec-dl-2019 results

dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019')

pt.Experiment(
    [pipeline_v2, pipeline_v2_hn, pipeline_v2_hnp],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG@10, R@1000], # Note: using R@1000 here instead of R(rel=2)@1000 to match the measure used by the TCT-ColBERT paper
    names=['v2', 'v2-hn', 'v2-hnp']
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2159/2159 [02:20<00:00, 15.37it/s]


,name,nDCG@10,R@1000
0,v2,0.685551,0.746504
1,v2-hn,0.709649,0.764716
2,v2-hnp,0.720577,0.761939


In [4]:
# dev/small results

dataset = pt.get_dataset('irds:msmarco-passage/dev/small')

pt.Experiment(
    [pipeline_v2, pipeline_v2_hn, pipeline_v2_hnp],
    dataset.get_topics(),
    dataset.get_qrels(),
    [RR@10, R@1000],
    names=['v2', 'v2-hn', 'v2-hnp']
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2159/2159 [24:59<00:00,  1.44it/s]


,name,RR@10,R@1000
0,v2,0.344800,0.967920
1,v2-hn,0.356027,0.971741
2,v2-hnp,0.359042,0.970010


You can speed up retrieval using `pyterrier_dr.TorchIndex` (processes on GPU), but note that results differ slightly due to differences & non-determinism in the order of floating point operations.